In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
import pickle

2024-05-28 16:58:50.889318: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 16:58:50.931908: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-28 16:58:50.933042: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 16:58:51.633038: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
data = pd.read_csv("../preprocessing/all-data-processed-3classes.csv")

In [4]:
data

,Sentiment,Sentence,Clean sentences
0,0,The utility will also provide services related...,utility also provide services related electric...
1,0,Niam offer financial investors a high return v...,niam offer financial investors high return via...
2,-1,Pinduoduo's stock plunges 23% premarket on hea...,pinduoduos stock plunges 23 premarket heavy vo...
3,0,Azeri Snap Elections Condemned by Monitors for...,azeri snap elections condemned monitors vote v...
4,-1,The #market is seeing strong #tax-loss selling...,market seeing strong taxloss selling investors...
...,...,...,...
31864,0,The U.S. Is Firing Blanks Against a New Irania...,yous firing blanks new iranian threat
31865,0,Storengy is the GDF SUEZ company that is dedic...,storengy gdf suez company dedicated undergroun...
31866,1,FDA approves Aquestive's ALS treatment https:...,fda approves aquestives als treatment httpstco...
31867,0,SoftBank to Create Japan Internet Giant to Bat...,softbank create japan internet giant battle gl...


In [5]:
data["Clean sentences"] = data["Clean sentences"].astype(str)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data["Clean sentences"], data["Sentiment"], test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((28682,), (3187,), (28682,), (3187,))

In [7]:
token = Tokenizer()
token.fit_on_texts(X_train)

In [8]:
vocab = len(token.index_word) + 1
print("Vocabulary size={}".format(len(token.word_index)))
print("Number of Documents={}".format(token.document_count))

Vocabulary size=29081
Number of Documents=28682


In [9]:
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [10]:
sequence_lengths = [len(seq) for seq in X_train]
max_length = max(sequence_lengths)
max_length

47

In [11]:
MAX_SEQUENCE_LENGTH = 47
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_train.shape, X_test.shape


((28682, 47), (3187, 47))

In [14]:
from tensorflow.keras.utils import to_categorical
y_train_labels = np.array(y_train) 
y_test_labels = np.array(y_test) 

In [15]:
y_train = np.where(y_train_labels == -1, 0, y_train_labels)
y_train = np.where(y_train_labels == 0, 1, y_train)
y_train = np.where(y_train_labels == 1, 2, y_train)

y_test = np.where(y_test_labels == -1, 0, y_test_labels)
y_test = np.where(y_test_labels == 0, 1, y_test)
y_test = np.where(y_test_labels == 1, 2, y_test)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [16]:
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.9

lr_scheduler = LearningRateScheduler(scheduler)

In [17]:
vec_size = 300
model = Sequential()
model.add(Embedding(len(token.index_word) + 1, vec_size, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))  # Output layer with softmax activation

model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

2024-05-28 17:01:03.226512: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-28 17:01:03.227906: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-28 17:01:03.228904: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 300)           8724600   
                                                                 
 lstm (LSTM)                 (None, 47, 128)           219648    
                                                                 
 dropout (Dropout)           (None, 47, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 8,993,851
Trainable params: 8,993,851
Non-

In [20]:
epochs = 100
batch_size = 4

# Define EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_lstm.h5', save_best_only=True, monitor='val_loss', mode='min')

# Train the model with callbacks
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping, model_checkpoint, lr_scheduler])

Epoch 1/20
897/897 [==============================] - ETA: 0s - loss: 1.0896 - accuracy: 0.3459

2024-05-28 17:05:32.732846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-28 17:05:32.734071: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-28 17:05:32.735082: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

897/897 [==============================] - 168s 187ms/step - loss: 1.0896 - accuracy: 0.3459 - val_loss: 0.9346 - val_accuracy: 0.5187 - lr: 1.0000e-04
Epoch 2/20
897/897 [==============================] - 160s 178ms/step - loss: 0.7399 - accuracy: 0.6465 - val_loss: 0.6227 - val_accuracy: 0.7568 - lr: 1.0000e-04
Epoch 3/20
897/897 [==============================] - 158s 176ms/step - loss: 0.3966 - accuracy: 0.8575 - val_loss: 0.3719 - val_accuracy: 0.8701 - lr: 1.0000e-04
Epoch 4/20
897/897 [==============================] - 161s 180ms/step - loss: 0.1898 - accuracy: 0.9413 - val_loss: 0.2840 - val_accuracy: 0.9078 - lr: 1.0000e-04
Epoch 5/20
897/897 [==============================] - 157s 175ms/step - loss: 0.1177 - accuracy: 0.9659 - val_loss: 0.2737 - val_accuracy: 0.9143 - lr: 1.0000e-04
Epoch 6/20
897/897 [==============================] - 163s 181ms/step - loss: 0.0816 - accuracy: 0.9773 - val_loss: 0.3103 - val_accuracy: 0.9084 - lr: 9.0000e-05
Epoch 7/20
897/897 [=============

In [21]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Get predicted probabilities
predictions = model.predict(X_test)

# Print some example predictions
print("Predicted probabilities:", predictions[:10])
print("Actual labels:", y_test[:10])

# Convert one-hot encoded y_test back to class labels for comparison
true_classes = np.argmax(y_test, axis=1)

# Create a DataFrame to compare predictions with actual values
results_df = pd.DataFrame({
    'Text': [" ".join([token.index_word.get(idx, "") for idx in x if idx != 0]) for x in X_test],
    'Actual Class': true_classes,
    'Predicted Class': np.argmax(predictions, axis=1),
    'Predicted Probabilities': list(predictions)
})

# Show the DataFrame
print(results_df.head(10))

100/100 [==============================] - 2s 24ms/step - loss: 0.2435 - accuracy: 0.9294
Test Loss: 0.24346762895584106
Test Accuracy: 0.9294006824493408


2024-05-28 17:31:12.561964: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-28 17:31:12.563160: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-28 17:31:12.564646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

100/100 [==============================] - 4s 32ms/step
Predicted probabilities: [[6.9997838e-04 9.9789977e-01 1.4002939e-03]
 [7.4439804e-04 3.5398430e-03 9.9571580e-01]
 [9.9748898e-01 1.3472395e-03 1.1637663e-03]
 [5.1357649e-04 9.9865609e-01 8.3035551e-04]
 [9.0883029e-01 5.6121849e-02 3.5047859e-02]
 [6.2489667e-04 9.9823070e-01 1.1444601e-03]
 [8.8266894e-04 5.3182505e-03 9.9379897e-01]
 [6.7392230e-04 2.5470846e-03 9.9677902e-01]
 [6.9846038e-04 2.6452783e-03 9.9665624e-01]
 [6.2078203e-04 9.9823773e-01 1.1414662e-03]]
Actual labels: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
                                                Text  Actual Class  \
0  international leads consumer gainers nio tata ...             1   
1         pleased bjorn wahlroos accepted nomination             2   
2  us energy information administration projects ...             0   
3  edited transcript earnings conference call pre...

In [22]:
results_df['sentiment_score'] = results_df['Predicted Probabilities'].apply(lambda x: -1 * x[0] + 1 * x[2])

In [23]:
results_df.to_excel("scores_lstm.xlsx")

In [24]:
model.save("sentimentModel_lstm.keras")

In [25]:
with open('tokenizer_lstm.pickle', 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)